# Architecture

### SVCNN and MVCNN
SVCNN.net_1 is only required. This will output a feature map of (512, 7, 7) after computation. 12 views would be (12, 512, 7, 7) = 25088. 

Then this is sent to MVCNN. net_1 of MVCNN can be removed as it is unnecessary. 

So 12xSVCNN (25088) -> MVCNN with an input size of 25088 so thats perfect

then MVCNN does the final classification

If we are to compute the gradient of each SVCNN, we'll check how much removing each SVCNN affects the loss, then they're ranked. 

if SVCNN_1 importance is 60%, theoritically we can prune 40% but idk if that's the best option but lets see

In [147]:
import os
import sys
parent_dir = os.path.abspath('../../MVCNN')
sys.path.append(parent_dir)

In [ ]:
import torch
from models import MVCNN

In [149]:
svcnn_weights = torch.load("/Users/mahadevsunilkumar/Desktop/DDNN-Working/src/MVCNN/trained_models/MVCNN_stage_1/MVCNN/model-00006.pth", map_location='mps', weights_only=False)

In [150]:
svcnn = MVCNN.SVCNN('svcnn')
svcnn.load_state_dict(svcnn_weights)

<All keys matched successfully>

In [151]:
mvcnn_weights = torch.load('/Users/mahadevsunilkumar/Desktop/DDNN-Working/src/MVCNN/trained_models/MVCNN_stage_2/MVCNN/model-00007.pth', map_location='mps', weights_only=False)

In [152]:
mvcnn = MVCNN.MVCNN('mvcnn', svcnn)

In [153]:
mvcnn.load_state_dict(mvcnn_weights)

<All keys matched successfully>

In [154]:
svcnn = svcnn.net_1
mvcnn = mvcnn.net_2
svcnn, mvcnn = svcnn.to('mps'), mvcnn.to('mps')
svcnn.eval()
mvcnn.eval()

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=40, bias=True)
)

In [155]:
svcnn_12 = {}

In [156]:
for i in range(12):
    svcnn_12[f'View_{i}'] = svcnn

In [157]:
x_svcnn = torch.randn(1, 3, 224, 224).to('mps')
x_svcnn

tensor([[[[ 6.2666e-01,  1.0340e+00, -1.5818e+00,  ...,  6.2205e-01,
            4.9044e-01, -1.4171e+00],
          [ 9.0482e-01, -5.8112e-01, -5.4249e-01,  ...,  7.5165e-03,
            1.0782e+00, -3.3952e-01],
          [ 4.1139e-01,  3.1233e-01, -4.0562e-01,  ..., -6.2831e-01,
           -2.5362e-01,  1.2244e+00],
          ...,
          [-4.9856e-01,  1.0945e+00,  9.5559e-01,  ..., -9.6129e-01,
            4.0098e-01,  1.9027e+00],
          [-2.9621e-03, -1.2129e+00,  1.5838e+00,  ..., -3.3259e-01,
           -2.8019e-01,  7.3189e-01],
          [-1.9951e+00,  4.5152e-01,  1.9030e-01,  ..., -8.5170e-02,
            2.6114e-02,  8.4217e-02]],

         [[ 1.9851e+00,  1.5599e+00,  1.9870e+00,  ..., -1.0362e+00,
           -8.4626e-01,  1.6317e+00],
          [-6.1954e-01, -1.4328e+00,  1.3768e-01,  ...,  1.9129e-01,
           -6.2494e-01,  3.4620e-03],
          [-1.1296e+00, -2.2089e-01, -9.9264e-01,  ..., -1.8576e-01,
            1.0527e+00, -5.2941e-01],
          ...,
     

In [158]:
x_svcnn.shape

torch.Size([1, 3, 224, 224])

In [165]:
tenor_4d = torch.randn(12, 512, 7, 7)
max_pooled = torch.max(tenor_4d, 0)[0].to('mps')
max_pooled = max_pooled.flatten().unsqueeze(0)
max_pooled.shape

torch.Size([1, 25088])

In [166]:
mvcnn(max_pooled)

tensor([[ -9.2765,  -6.3426,  -3.5377,  -6.2333, -10.2069, -10.2224,  -7.3638,
         -10.3639,  -6.5267,  -7.3924,  -4.9836,  -6.6519,  -2.5672, -12.2452,
          -6.6258,  -3.0855,  -8.5797,  -7.9177, -14.1395,  -2.5575,  -4.7683,
         -10.2511,  -4.4620,  -2.2969,  -8.0255,  -4.0723,  -3.7950, -10.3833,
          -9.7041,  -8.8675,  -9.4005,  -4.8046,  -5.7892,   1.8220,  -5.1677,
         -10.6601,  -7.5772,  -3.0590, -12.4178, -11.0302]], device='mps:0',
       grad_fn=<LinearBackward0>)